# Tutorial 7: Classification (Part II)

### Lecture and Tutorial Learning Goals:

After completing this week's lecture and tutorial work, you will be able to:

* Describe what a test data set is and how it is used in classification.
* Using Python, evaluate classification accuracy using a test data set and appropriate metrics.
* Using Python, execute cross validation in Python to choose the number of neighbours.
* Identify when it is necessary to scale variables before classification and do this using Python
* In a dataset with > 2 attributes, perform k-nearest neighbour classification in Python using the `scikit-learn` package to predict the class of a test dataset.
* Describe advantages and disadvantages of the k-nearest neighbour classification algorithm.


## Handwritten Digit Classification using Python

<img src="https://media.giphy.com/media/UwrdbvJz1CNck/giphy.gif" width = "600"/>

Source: https://media.giphy.com/media/UwrdbvJz1CNck/giphy.gif

MNIST is a computer vision dataset that consists of images of handwritten digits like these:

![](img/MNIST.png)

It also includes labels for each image, telling us which digit it is. For example, the labels for the above images are 5, 0, 4, and 1.


In this tutorial, we’re going to train a classifier to look at images and predict what digits they are. Our goal isn’t to train a really elaborate model that achieves state-of-the-art performance, but rather to dip a toe into using classification with pixelated images. As such, we’re going to keep working with the simple K-nearest neighbour classifier we have been exploring in the last two weeks.



### Using image data for classification

As mentioned earlier, every MNIST data point has two parts: an image of a handwritten digit and a corresponding label. Both the training set and test set contain images and their corresponding labels.

Each image is 28 pixels by 28 pixels. We can interpret this as a big matrix of numbers:

<img src="img/MNIST-Matrix.png" width = "500"/>


We can flatten this matrix into a vector of 28x28 = 784 numbers and give it a class label (here 1 for the number one). It doesn’t matter how we flatten the array, as long as we’re consistent between images. From this perspective, the MNIST images are just a bunch of points in a 784-dimensional vector space, with a very rich structure.

<img src="img/matrix_to_row.png" width = "1000"/>


We do this for every image of the digits we have, and we create a data table like the one shown below that we can use for classification. Note, like any other classification problem that we have seen before, we need many observations for each class. This problem is also a bit different from the first classification problem we have encountered (Wisonsin breast cancer data set), in that we have more than two classes (here we have 10 classes, one for each digit from 0 to 9).

<img src="img/data_table.png" width = "700"/>

This information is taken from: https://tensorflow.rstudio.com/tensorflow/articles/tutorial_mnist_beginners.html

In [ ]:
### Run this cell before continuing.

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier


# functions needed to work with images
# code below sourced from: https://gist.github.com/daviddalpiaz/ae62ae5ccd0bada4b9acd6dbc9008706
# helper function for visualization
def show_digit(arr784):
    plt.imshow(np.array(arr784)[1:].reshape(28, 28), cmap="gray")

**Question 1.0** Multiple Choice:
<br> {points: 1}

How many rows and columns does the array of an image have?

A. 784 columns and 1 row

B. 28 columns and 1 row

C. 18 columns and 18 rows

D. 28 columns and 28 rows 

*Assign your answer to an object called `answer1_0`. Make sure the correct answer is an uppercase letter and to surround your answer with quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_0)).encode("utf-8")+b"a7a881512442ba7d").hexdigest() == "5cf511062016ef7ef7a56a7a521ebcbcd0e42ab0", "type of answer1_0 is not str. answer1_0 should be an str"
assert sha1(str(len(answer1_0)).encode("utf-8")+b"a7a881512442ba7d").hexdigest() == "73adbd15992cad5156a54f13e39953e75407160a", "length of answer1_0 is not correct"
assert sha1(str(answer1_0.lower()).encode("utf-8")+b"a7a881512442ba7d").hexdigest() == "d2bf9e838712cfea48b16ac658fd19cd8b63a3ef", "value of answer1_0 is not correct"
assert sha1(str(answer1_0).encode("utf-8")+b"a7a881512442ba7d").hexdigest() == "e4c51b572690d8f4a259ef6b8328d63fd748a5d1", "correct string value of answer1_0 but incorrect case of letters"

print('Success!')

**Question 1.1** Multiple Choice: 
<br> {points: 1}

Once we linearize the array, how many rows represent a number?

A. 28

B. 784

C. 1

D. 18

*Assign your answer to an object called `answer1_1`. Make sure the correct answer is an uppercase letter and to surround your answer with quotation marks (e.g. `"F"`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer1_1)).encode("utf-8")+b"30963ba55196edc8").hexdigest() == "fe3a80c00d0885de2fef20c3c58ca05114f8b845", "type of answer1_1 is not str. answer1_1 should be an str"
assert sha1(str(len(answer1_1)).encode("utf-8")+b"30963ba55196edc8").hexdigest() == "fb8cf4b2f60e2f14784209bf35cc5fb0b0e75c7e", "length of answer1_1 is not correct"
assert sha1(str(answer1_1.lower()).encode("utf-8")+b"30963ba55196edc8").hexdigest() == "42e1ea5655998fa934a8fcdab548ff343fc0156d", "value of answer1_1 is not correct"
assert sha1(str(answer1_1).encode("utf-8")+b"30963ba55196edc8").hexdigest() == "fcabd670ae1d84522b78d4ddf8749cde923a1c51", "correct string value of answer1_1 but incorrect case of letters"

print('Success!')

## 2. Exploring the Data

Before we move on to do the modeling component, it is always required that we take a look at our data and understand the problem and the structure of the data well. We can start this part by loading the images and taking a look at the first rows of the dataset. You can load the data set by running the cell below. 

In [ ]:
# Load images.
# Run this cell.
training_data = pd.read_csv("data/mnist_train_small.csv")
testing_data = pd.read_csv("data/mnist_test_small.csv")

Look at the first 6 rows of `training_data`. What do you notice?

In [ ]:
training_data.head(6)

In [ ]:
training_data.shape

There are no class labels! This data set has already been split into the X's (which you loaded above) and the labels. In addition, there is an extra "X" column which represents the row number (1, 2, 3...). **Keep this in mind for now because we will remove it later on.** Now, let's load the labels.

In [ ]:
# Run this cell.
training_labels = pd.read_csv("data/mnist_train_label_small.csv")["y"]
testing_labels = pd.read_csv("data/mnist_test_label_small.csv")["y"]

Look at the first 6 labels of `training_labels` using the `data_frame.head(6)` function. 

In [ ]:
# Use this cell to view the first 6 labels.
# Run this cell.
training_labels.head(6)

In [ ]:
testing_labels.head(6)

**Question 2.0**
<br> {points: 1}

How many rows does the training data set have? Note, each row is a different number in the postal code system. 

Use `len`. Note, the testing data set should have fewer rows than the training data set.

*Assign your answer to an object called `number_of_rows`. Make sure your answer is a numeric and so it should not be surrounded by quotation marks.*

In [ ]:
# your code here
raise NotImplementedError
number_of_rows

In [ ]:
from hashlib import sha1
assert sha1(str(type(number_of_rows)).encode("utf-8")+b"b8b0923fc85be6fe").hexdigest() == "20d5625e774b34b1428de962fd6bf92ee5224665", "type of number_of_rows is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(number_of_rows).encode("utf-8")+b"b8b0923fc85be6fe").hexdigest() == "db11005193d7deeb832896a64f2391814d91147c", "value of number_of_rows is not correct"

print('Success!')

**Question 2.1**
<br> {points: 1}

For mutli-class classification with k-nn it is important for the classes to have about the same number of observations in each class. For example, if  90% of our training set observationas were labeled as 2's, then k-nn classification predict 2 almost every time and we would get an accuracy score of 90% even though our classifier wasn't really doing a great job. 

Use the `value_counts` function to get the counts for each group and see if the data set is balanced across the classes (has roughly equal numbers of observation for each class). Name the output `counts`. 

In [ ]:
# your code here
raise NotImplementedError
counts

In [ ]:
from hashlib import sha1
assert sha1(str(type(len(counts))).encode("utf-8")+b"063355771d43001f").hexdigest() == "9afaf3a7ca687a87335a5701cecb46071cb3f9dd", "type of len(counts) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(len(counts)).encode("utf-8")+b"063355771d43001f").hexdigest() == "ace6c9faf12d0e80e6e0ece3f7736740dc6cdd60", "value of len(counts) is not correct"

assert sha1(str(type(sum(counts.values))).encode("utf-8")+b"7f2a0697bca08fea").hexdigest() == "51f100525662e4a45f5f2cad783c32a953837f6b", "type of sum(counts.values) is not correct"
assert sha1(str(sum(counts.values)).encode("utf-8")+b"7f2a0697bca08fea").hexdigest() == "178e6e5ee4463b8eb8dd578668c7e224589ec95f", "value of sum(counts.values) is not correct"

assert sha1(str(type(pd.DataFrame(counts).reset_index().shape)).encode("utf-8")+b"927743044191eccf").hexdigest() == "0735942b10cd0e876f47f702b366c0ba7693680b", "type of pd.DataFrame(counts).reset_index().shape is not tuple. pd.DataFrame(counts).reset_index().shape should be a tuple"
assert sha1(str(len(pd.DataFrame(counts).reset_index().shape)).encode("utf-8")+b"927743044191eccf").hexdigest() == "686c5d3eca25e33ebc8c1e2492f667d1ddcdf36f", "length of pd.DataFrame(counts).reset_index().shape is not correct"
assert sha1(str(sorted(map(str, pd.DataFrame(counts).reset_index().shape))).encode("utf-8")+b"927743044191eccf").hexdigest() == "d860ba13e2efaf9fc2f8828616ab2c3cdc62eaf8", "values of pd.DataFrame(counts).reset_index().shape are not correct"
assert sha1(str(pd.DataFrame(counts).reset_index().shape).encode("utf-8")+b"927743044191eccf").hexdigest() == "7d517ce3ed0c69215696821fda2d47a181d62e02", "order of elements of pd.DataFrame(counts).reset_index().shape is not correct"

assert sha1(str(type(sum(pd.DataFrame(counts).reset_index().index))).encode("utf-8")+b"7a4fed96fdf9ced7").hexdigest() == "a513327a68d75932bca94fb4fdda963d1d7683e6", "type of sum(pd.DataFrame(counts).reset_index().index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(pd.DataFrame(counts).reset_index().index)).encode("utf-8")+b"7a4fed96fdf9ced7").hexdigest() == "13a1adf9bb17d331d0a631a0e2c223ed7ea92983", "value of sum(pd.DataFrame(counts).reset_index().index) is not correct"

print('Success!')

**Question 2.2** True or False:
<br> {points: 1}

The classes are not balanced. Some of them are *many times* larger or smaller than others. 

*Assign your answer to an object called `answer2_2`. Make sure your answer is a boolean. i.e. `True` or `False`.*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer2_2)).encode("utf-8")+b"afb04cc83cee3393").hexdigest() == "648f4f9d70c37a9d2fa5dad9933b0b08c9e29329", "type of answer2_2 is not bool. answer2_2 should be a bool"
assert sha1(str(answer2_2).encode("utf-8")+b"afb04cc83cee3393").hexdigest() == "e5457b2d8a87b1eb25c2496f2fdbfd2e0d866c8e", "boolean value of answer2_2 is not correct"

print('Success!')

To view an image in the notebook, you can use the `show_digit` function (we gave you the code for this function in the first code cell in the notebook, All you have to do to use it is run the cell below). The `show_digit` function takes the row from the dataset whose image you want to produce, which you can obtain using the `iloc` method.

The code we provide below will show you the image for the observation in the 200th row from the training data set. 

In [ ]:
# Run this cell to get the images for the 200th row from the training data set.
show_digit(training_data.iloc[199])

**Question 2.3** 
<br> {points: 3}

Show the image for row 102.

In [ ]:
# your code here
raise NotImplementedError

If you are unsure as to what number the plot is depicting (because the handwriting is messy) you can get the label from the `training_labels`:

In [ ]:
# run this cell to get the training label for the 200th row
training_labels[199]

**Question 2.4** 
<br> {points: 1}

What is the class label for row 102? 

*Assign your answer to an object called `label_102`.*

In [ ]:
# your code here
raise NotImplementedError
label_102

In [ ]:
from hashlib import sha1
assert sha1(str(type(label_102)).encode("utf-8")+b"3ab65a46d6491f62").hexdigest() == "be60b9e8daa83bf000e82a34461ceace8eadf34d", "type of label_102 is not correct"
assert sha1(str(label_102).encode("utf-8")+b"3ab65a46d6491f62").hexdigest() == "0513ef0586440837fe2382a2782e6ce23b9c7cbf", "value of label_102 is not correct"

print('Success!')

## 3. Splitting the Data

**Question 3.1**
<br> {points: 3}

We have already split the data into two datasets, one for training purposes and one for testing purposes. **Is it important to split the data into a training and testing dataset when designing a knn classification model?** If yes, why do we do this? If no, explain why this is not a good idea.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## Which $k$ should we use?

As you learned from the worksheet, we can use cross-validation on the training data set to select which $k$ is the most optimal for our data set for k-nn classification. 

**Question 3.2**
<br> {points: 1}

To get all the marks in this question, you will have to:
- Create a parameter grid that includes a set of ranges for the hyperparameter that needs to be tuned
- Perform a 5-fold cross-validation on the training set
- Create a grid search model with your `KNeighborsClassifier` and a parameter grid (covering `n_neighbors` from 2 to 14, both inclusive), and *specify `return_train_score=True` and `n_jobs=-1`.* 
- Fit the grid search model to training set
- Get the `cv_results_` from the fitted model
- Plot the $k$ vs the accuracy
    - Assign this plot to an object called `cross_val_plot`

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(1234)

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(cross_val_plot is None)).encode("utf-8")+b"c99d7dda168a5525").hexdigest() == "4446d6dd8fb0649e08c1daf2730fa9886b8c81d4", "type of cross_val_plot is None is not bool. cross_val_plot is None should be a bool"
assert sha1(str(cross_val_plot is None).encode("utf-8")+b"c99d7dda168a5525").hexdigest() == "4089db20afde90bc4b3baa6d862289b611c9651b", "boolean value of cross_val_plot is None is not correct"

assert sha1(str(type(cross_val_plot.mark.type)).encode("utf-8")+b"b8f2826b121e7759").hexdigest() == "be25d2f0ae9d836b4f375fc78d3905f2a3830d03", "type of cross_val_plot.mark.type is not str. cross_val_plot.mark.type should be an str"
assert sha1(str(len(cross_val_plot.mark.type)).encode("utf-8")+b"b8f2826b121e7759").hexdigest() == "f096b71acfb8f955d41bbea90b94cebe44563924", "length of cross_val_plot.mark.type is not correct"
assert sha1(str(cross_val_plot.mark.type.lower()).encode("utf-8")+b"b8f2826b121e7759").hexdigest() == "461a11299491197194c938ebdce58a6fd58564f8", "value of cross_val_plot.mark.type is not correct"
assert sha1(str(cross_val_plot.mark.type).encode("utf-8")+b"b8f2826b121e7759").hexdigest() == "461a11299491197194c938ebdce58a6fd58564f8", "correct string value of cross_val_plot.mark.type but incorrect case of letters"

assert sha1(str(type(cross_val_plot.mark['point'])).encode("utf-8")+b"8bc82201924332e6").hexdigest() == "a67f7cbdfe0e949b124e0db1df548a36217a603c", "type of cross_val_plot.mark['point'] is not bool. cross_val_plot.mark['point'] should be a bool"
assert sha1(str(cross_val_plot.mark['point']).encode("utf-8")+b"8bc82201924332e6").hexdigest() == "0e6ba18a22eadaa5f62e2f4e0bbf7e2897974c98", "boolean value of cross_val_plot.mark['point'] is not correct"

assert sha1(str(type(cross_val_plot.encoding.x.field)).encode("utf-8")+b"d5f7d13970689729").hexdigest() == "8086899f670368cc56e74bcb39d480342ddb1430", "type of cross_val_plot.encoding.x.field is not str. cross_val_plot.encoding.x.field should be an str"
assert sha1(str(len(cross_val_plot.encoding.x.field)).encode("utf-8")+b"d5f7d13970689729").hexdigest() == "70f85be90bf4028be68efbc66e99ce6c403a6af0", "length of cross_val_plot.encoding.x.field is not correct"
assert sha1(str(cross_val_plot.encoding.x.field.lower()).encode("utf-8")+b"d5f7d13970689729").hexdigest() == "ba0043a29b8e865891629261541e17b1dba5c322", "value of cross_val_plot.encoding.x.field is not correct"
assert sha1(str(cross_val_plot.encoding.x.field).encode("utf-8")+b"d5f7d13970689729").hexdigest() == "ba0043a29b8e865891629261541e17b1dba5c322", "correct string value of cross_val_plot.encoding.x.field but incorrect case of letters"

assert sha1(str(type(cross_val_plot.encoding.y.field)).encode("utf-8")+b"fec3f20e9a114d70").hexdigest() == "9c1c69058dd11e109c5f030fd3737f41032e81a2", "type of cross_val_plot.encoding.y.field is not str. cross_val_plot.encoding.y.field should be an str"
assert sha1(str(len(cross_val_plot.encoding.y.field)).encode("utf-8")+b"fec3f20e9a114d70").hexdigest() == "2b42ccb55cffbb4a1612244db065524c203e1b13", "length of cross_val_plot.encoding.y.field is not correct"
assert sha1(str(cross_val_plot.encoding.y.field.lower()).encode("utf-8")+b"fec3f20e9a114d70").hexdigest() == "6e672b7c5cef91a15f3a9c41f2643319d4214d01", "value of cross_val_plot.encoding.y.field is not correct"
assert sha1(str(cross_val_plot.encoding.y.field).encode("utf-8")+b"fec3f20e9a114d70").hexdigest() == "6e672b7c5cef91a15f3a9c41f2643319d4214d01", "correct string value of cross_val_plot.encoding.y.field but incorrect case of letters"

assert sha1(str(type(sum(cross_val_plot.data.mean_train_score))).encode("utf-8")+b"ae1761c6c15e1ede").hexdigest() == "6d0d4e8f70c01adac0e9c2aa8bb68e3569495739", "type of sum(cross_val_plot.data.mean_train_score) is not float. Please make sure it is float and not np.float64, etc. You can cast your value into a float using float()"
assert sha1(str(round(sum(cross_val_plot.data.mean_train_score), 2)).encode("utf-8")+b"ae1761c6c15e1ede").hexdigest() == "19594f4d3646f5342059e4d9358ba0f25977da3b", "value of sum(cross_val_plot.data.mean_train_score) is not correct (rounded to 2 decimal places)"

assert sha1(str(type(sum(cross_val_plot.data.param_n_neighbors))).encode("utf-8")+b"c9dc5ce7bc65f22b").hexdigest() == "b753d467b1853addc022bcd1c4d49abb10aaeb5f", "type of sum(cross_val_plot.data.param_n_neighbors) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(cross_val_plot.data.param_n_neighbors)).encode("utf-8")+b"c9dc5ce7bc65f22b").hexdigest() == "fdb996982a3ece38077518f33f94f2a89707352b", "value of sum(cross_val_plot.data.param_n_neighbors) is not correct"

print('Success!')

**Question 3.3**
<br> {points: 3}

Based on the plot from **Question 3.2**, which $k$ would you choose and how can you be sure about your decision? In your answer you should reference why we do cross-validation.

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.

## 4. Let's build our model

**Question 4.0**
<br> {points: 3}

Now that we have explored our data, separated the data into training and testing sets (was technically done for you), and applied cross-validation to choose the best $k$, we can build our final model.

First, build your model specification with the best value for $K$. Assign your answer to an object called `mnist_spec`.

Then, pass the model specification and the training data set to the `fit` function. 

*Assign your answer to an object called `mnist_fit`.*

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(9999)

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(mnist_spec is None)).encode("utf-8")+b"b014e3f58045aabf").hexdigest() == "c7d1c64fe0bb477323556c1f98b7d90279d6d0d8", "type of mnist_spec is None is not bool. mnist_spec is None should be a bool"
assert sha1(str(mnist_spec is None).encode("utf-8")+b"b014e3f58045aabf").hexdigest() == "93294573829bca41204d2b518d1aa5eb0906f10a", "boolean value of mnist_spec is None is not correct"

assert sha1(str(type(mnist_fit is None)).encode("utf-8")+b"8100c84b54f270d3").hexdigest() == "692fc137d180c480c235e85667ca5d9aeb2952d2", "type of mnist_fit is None is not bool. mnist_fit is None should be a bool"
assert sha1(str(mnist_fit is None).encode("utf-8")+b"8100c84b54f270d3").hexdigest() == "73c0fae619a93c94a1d520a9567078efed1de643", "boolean value of mnist_fit is None is not correct"

assert sha1(str(type(mnist_fit.n_neighbors)).encode("utf-8")+b"f2acc9b9969db4a6").hexdigest() == "03bb5cbba37dbcc5b9733bfe6d2c8d9f91f2e853", "type of mnist_fit.n_neighbors is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(mnist_fit.n_neighbors).encode("utf-8")+b"f2acc9b9969db4a6").hexdigest() == "59c51e925809ca25b211cb400d998d8840eb768b", "value of mnist_fit.n_neighbors is not correct"

assert sha1(str(type(mnist_fit.weights)).encode("utf-8")+b"ce48484c9f33f9e4").hexdigest() == "8db7b60e248f0c393fc3e48bb5019ae93ca63b0b", "type of mnist_fit.weights is not str. mnist_fit.weights should be an str"
assert sha1(str(len(mnist_fit.weights)).encode("utf-8")+b"ce48484c9f33f9e4").hexdigest() == "852a7beda01bf9dfcf9b1d5bf16e4dfdb34940ae", "length of mnist_fit.weights is not correct"
assert sha1(str(mnist_fit.weights.lower()).encode("utf-8")+b"ce48484c9f33f9e4").hexdigest() == "199bba23af8a27e3da79e8aa082f2fd59abb256f", "value of mnist_fit.weights is not correct"
assert sha1(str(mnist_fit.weights).encode("utf-8")+b"ce48484c9f33f9e4").hexdigest() == "199bba23af8a27e3da79e8aa082f2fd59abb256f", "correct string value of mnist_fit.weights but incorrect case of letters"

assert sha1(str(type(type(mnist_fit))).encode("utf-8")+b"40be7add16ba97a5").hexdigest() == "e9dbcbe900fb4ce3ed9e7aaa13e24f5a91acb9f4", "type of type(mnist_fit) is not correct"
assert sha1(str(type(mnist_fit)).encode("utf-8")+b"40be7add16ba97a5").hexdigest() == "cde1096167ceec4368aae91c8b492c20e2441b38", "value of type(mnist_fit) is not correct"

assert sha1(str(type(mnist_fit.n_samples_fit_)).encode("utf-8")+b"79b3b578db2916e5").hexdigest() == "100f78c806ee2bece401edf173b35eb300e0070b", "type of mnist_fit.n_samples_fit_ is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(mnist_fit.n_samples_fit_).encode("utf-8")+b"79b3b578db2916e5").hexdigest() == "b9aa065ff6cd7348fe4f17ee82d7c2939efa984d", "value of mnist_fit.n_samples_fit_ is not correct"

print('Success!')

**Question 4.1**
<br> {points: 1}

Use your final model to predict on the test dataset and assign this to an object called `mnist_predictions`. Combine the `testing_data`, `mnist_predictions` and `testing_labels` into one dataframe using `pd.concat` function and assign it to an object called `mnist_df`. Report the accuracy of this prediction, and store this in an object named `mnist_metrics`. Also report the confusion matrix and and store this in an object named `mnist_conf_mat`. 

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(9999)

# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert str(type(mnist_predictions is None)) == "<class 'bool'>", "type of mnist_predictions is None is not bool. mnist_predictions is None should be a bool"
assert str(mnist_predictions is None) == "False", "boolean value of mnist_predictions is None is not correct"

assert str(type(sum(mnist_predictions))) == "<class 'numpy.int64'>", "type of sum(mnist_predictions) is not correct"
assert str(sum(mnist_predictions)) == "2548", "value of sum(mnist_predictions) is not correct"

assert str(type(sum(mnist_df.y))) == "<class 'int'>", "type of sum(mnist_df.y) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert str(sum(mnist_df.y)) == "2638", "value of sum(mnist_df.y) is not correct"

assert str(type(sum(mnist_df.predicted))) == "<class 'int'>", "type of sum(mnist_df.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert str(sum(mnist_df.predicted)) == "2548", "value of sum(mnist_df.predicted) is not correct"

assert str(type(mnist_conf_mat is None)) == "<class 'bool'>", "type of mnist_conf_mat is None is not bool. mnist_conf_mat is None should be a bool"
assert str(mnist_conf_mat is None) == "False", "boolean value of mnist_conf_mat is None is not correct"

assert str(type(mnist_conf_mat)) == "<class 'numpy.ndarray'>", "type of type(mnist_conf_mat) is not correct"

assert str(type(mnist_conf_mat.sum())) == "<class 'numpy.int64'>", "type of mnist_conf_mat.sum() is not correct"
assert str(mnist_conf_mat.sum()) == "600", "value of mnist_conf_mat.sum() is not correct"

assert str(type(mnist_metrics is None)) == "<class 'bool'>", "type of mnist_metrics is None is not bool. mnist_metrics is None should be a bool"
assert str(mnist_metrics is None) == "False", "boolean value of mnist_metrics is None is not correct"

assert str(type(mnist_metrics)) == "<class 'numpy.float64'>", "type of type(mnist_metrics) is not correct"

assert str(type(mnist_metrics)) == "<class 'numpy.float64'>", "type of mnist_metrics is not correct"
assert str(mnist_metrics) == "0.7666666666666667", "value of mnist_metrics is not correct"

print('Success!')

**Question 4.2**
<br> {points: 3}

For this exercise, print out 3 images and the true labels from the test set that were predicted correctly. 

To approach this exercise, we will first create a data frame that contains the predictions, and the labels from the testing set. We will want to use this data to find cases where the hand written digits were predicted correctly (i.e., any rows where the values in the predicted class label column, `predicted`, match the test set labels, `y`). We will then want to use this information to go back to the original data and extract images that correspond to these correct predictions. 

To keep track of which rows correspond to which images, we will use `reset_index` to reset the index of the dataframe to get the row numbers of the dataframe. It will be useful to keep track of which rows correspond to which images when we do something to the data frame, for example, filter it to find correctly predicted labels. Scaffolding has been provided below for you. *Assign your answer to an object called `mnist_predictions_with_labels`.*

From this data frame, filter for cases of equality between the predictions, `predicted`, and testing set labels, `y`. Essentially, you want to find cases where the predictions match the testing set labels. Sample 3 rows of data using `head`. *Assign your answer to an object called `matching`.* 

Next, we want to extract the row numbers (`index`) from the `matching` data frame to input them into the `show_digit` function. The scaffolding to extract the data from the first row and the associated row number is provided below. *Assign your answers respectively to `matching_1`, `matching_2`, and `matching_3`.*

Finally, utilize the `show_digit` function to print our your images by inputting the sliced testing data set where the predictions match the testing set labels (`matching_1`, `matching_2`, `matching_3`).  

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(1000)

# ___ = mnist_df.reset_index()[
#     ["index", ___, ___]
# ]

# ___ = ___[
#     mnist_predictions_with_labels[___] == mnist_predictions_with_labels[___]
# ].head(___)

# ___ = ___.iloc[___.iloc[0, 0]] # the first row and the first column
# ___ = ___.iloc[___.iloc[__, 0]] # the second row and the first column
# ___ = ___.iloc[___.iloc[__, 0]] # the third row and the first column

# your code here
raise NotImplementedError

In [ ]:
show_digit(matching_1)

In [ ]:
show_digit(matching_2)

In [ ]:
show_digit(matching_3)

In [ ]:
from hashlib import sha1
assert sha1(str(type(mnist_predictions_with_labels is None)).encode("utf-8")+b"f7222c65cfb2a0e8").hexdigest() == "f9e26c95ba68c0477c3bdb21054739e8aedea435", "type of mnist_predictions_with_labels is None is not bool. mnist_predictions_with_labels is None should be a bool"
assert sha1(str(mnist_predictions_with_labels is None).encode("utf-8")+b"f7222c65cfb2a0e8").hexdigest() == "5abab6fced6af05e6cf51275f4617ca4577b028d", "boolean value of mnist_predictions_with_labels is None is not correct"

assert sha1(str(type(matching is None)).encode("utf-8")+b"9a2d0800c698d8f6").hexdigest() == "57bd7ac418ff3cdf4763ad237f6c4f11f2b2ef05", "type of matching is None is not bool. matching is None should be a bool"
assert sha1(str(matching is None).encode("utf-8")+b"9a2d0800c698d8f6").hexdigest() == "28bfebe8e2e7effd06d12e4ca60cb5bb003c8012", "boolean value of matching is None is not correct"

assert sha1(str(type(matching_1 is None)).encode("utf-8")+b"9481a309b4440158").hexdigest() == "e997d5789f4d9aa9f6883ce6611607369c65554c", "type of matching_1 is None is not bool. matching_1 is None should be a bool"
assert sha1(str(matching_1 is None).encode("utf-8")+b"9481a309b4440158").hexdigest() == "87509b404a1523c43bce5b133afd2e0e9a497bb8", "boolean value of matching_1 is None is not correct"

assert sha1(str(type(matching_2 is None)).encode("utf-8")+b"2f8e49ef39b156f7").hexdigest() == "e0dba8d57f0a92beb30b69d96f304a8eafdc770b", "type of matching_2 is None is not bool. matching_2 is None should be a bool"
assert sha1(str(matching_2 is None).encode("utf-8")+b"2f8e49ef39b156f7").hexdigest() == "f7198fadb4424a5462c3af0533ccbf91dfd10931", "boolean value of matching_2 is None is not correct"

assert sha1(str(type(matching_3 is None)).encode("utf-8")+b"228cdd2455944d88").hexdigest() == "cf876a143d3f4a7756446fca636664dc16589417", "type of matching_3 is None is not bool. matching_3 is None should be a bool"
assert sha1(str(matching_3 is None).encode("utf-8")+b"228cdd2455944d88").hexdigest() == "d3e79b33ea60001a7b9aa18ad40867cd18c3d4a9", "boolean value of matching_3 is None is not correct"

assert sha1(str(type(mnist_predictions_with_labels)).encode("utf-8")+b"4384b1a3cfe82345").hexdigest() == "bb1d469309fa4ecb6e60165d3ba0ca915fa713f3", "type of type(mnist_predictions_with_labels) is not correct"

assert sha1(str(type('index' in mnist_predictions_with_labels.columns)).encode("utf-8")+b"2e6587d693c542c8").hexdigest() == "cb1a23153289973d24aba4607389a102974b0192", "type of 'index' in mnist_predictions_with_labels.columns is not bool. 'index' in mnist_predictions_with_labels.columns should be a bool"
assert sha1(str('index' in mnist_predictions_with_labels.columns).encode("utf-8")+b"2e6587d693c542c8").hexdigest() == "ecf6802232bb5b667e42a4f0dfc1cea8d49928df", "boolean value of 'index' in mnist_predictions_with_labels.columns is not correct"

assert sha1(str(type(mnist_predictions_with_labels.shape)).encode("utf-8")+b"c7c896327cca1565").hexdigest() == "0ec20f02352523cd2c11a98a9b4baf1e95666172", "type of mnist_predictions_with_labels.shape is not tuple. mnist_predictions_with_labels.shape should be a tuple"
assert sha1(str(len(mnist_predictions_with_labels.shape)).encode("utf-8")+b"c7c896327cca1565").hexdigest() == "d67a51211311fe4027c54937cfbd63cf6aa4e09c", "length of mnist_predictions_with_labels.shape is not correct"
assert sha1(str(sorted(map(str, mnist_predictions_with_labels.shape))).encode("utf-8")+b"c7c896327cca1565").hexdigest() == "7765aa3170aab8751cfc2928bd1105871c98dd42", "values of mnist_predictions_with_labels.shape are not correct"
assert sha1(str(mnist_predictions_with_labels.shape).encode("utf-8")+b"c7c896327cca1565").hexdigest() == "6049d0024171cce38ab272e2dfaf037c098cb429", "order of elements of mnist_predictions_with_labels.shape is not correct"

assert sha1(str(type(sum(mnist_predictions_with_labels.predicted))).encode("utf-8")+b"ec4042a1ecd6d1ca").hexdigest() == "951137efbac04aa0472cb7a3ba4e6e8ae7c751ec", "type of sum(mnist_predictions_with_labels.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(mnist_predictions_with_labels.predicted)).encode("utf-8")+b"ec4042a1ecd6d1ca").hexdigest() == "3166f64a1a68099119b157ce75e118840c45a7b3", "value of sum(mnist_predictions_with_labels.predicted) is not correct"

assert sha1(str(type(sum(mnist_predictions_with_labels.index))).encode("utf-8")+b"fed069fa12c302a0").hexdigest() == "6ab3837b4fe3259c31581b85ae0ec2bf9121b25b", "type of sum(mnist_predictions_with_labels.index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(mnist_predictions_with_labels.index)).encode("utf-8")+b"fed069fa12c302a0").hexdigest() == "739f7b24c4f260b66e55045ddd1f1771de1b80cb", "value of sum(mnist_predictions_with_labels.index) is not correct"

assert sha1(str(type(matching)).encode("utf-8")+b"99e95c1cd49c0625").hexdigest() == "1fae8603326a8905c8159b8b4665e555d836cb00", "type of type(matching) is not correct"

assert sha1(str(type(matching.shape)).encode("utf-8")+b"307df37426450d91").hexdigest() == "aea58565b39889071218567b2d9c181ca5bcb70b", "type of matching.shape is not tuple. matching.shape should be a tuple"
assert sha1(str(len(matching.shape)).encode("utf-8")+b"307df37426450d91").hexdigest() == "9f804ce0450fe310366773eece2f8f1711ee3b42", "length of matching.shape is not correct"
assert sha1(str(sorted(map(str, matching.shape))).encode("utf-8")+b"307df37426450d91").hexdigest() == "6a2d5d3b5a941a0901cfdea31227d8aa76c282a7", "values of matching.shape are not correct"
assert sha1(str(matching.shape).encode("utf-8")+b"307df37426450d91").hexdigest() == "7fd47d8fc65ae02c33ce32fcf0492e6fa74fa652", "order of elements of matching.shape is not correct"

assert sha1(str(type(sum(matching.predicted))).encode("utf-8")+b"71afd83ee9644441").hexdigest() == "4c356aa375ecc1f6f7f8aa03e2329b2c4db12827", "type of sum(matching.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(matching.predicted)).encode("utf-8")+b"71afd83ee9644441").hexdigest() == "f7a760bac2ff86fe064e2343a34e1cc86d22d78d", "value of sum(matching.predicted) is not correct"

assert sha1(str(type(sum(matching.index))).encode("utf-8")+b"110f053f50235fce").hexdigest() == "bcb25cf2063082d6d190d464c6af329094bbc407", "type of sum(matching.index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(matching.index)).encode("utf-8")+b"110f053f50235fce").hexdigest() == "7118eb1ce455f01432591cfa06afb623171e2b1d", "value of sum(matching.index) is not correct"

assert sha1(str(type(matching_1)).encode("utf-8")+b"41a2d24bfb4a1f94").hexdigest() == "d49c0cd92259efe21bb3ae0242b534323a810bb3", "type of matching_1 is not correct"
assert sha1(str(matching_1).encode("utf-8")+b"41a2d24bfb4a1f94").hexdigest() == "19758ef31a8fabb47b06a4b80417db9955cd49e5", "value of matching_1 is not correct"

assert sha1(str(type(matching_2)).encode("utf-8")+b"ef442a5e55482e54").hexdigest() == "9a3b0b61af20910aff39644c3338e2ee6ceee3c8", "type of matching_2 is not correct"
assert sha1(str(matching_2).encode("utf-8")+b"ef442a5e55482e54").hexdigest() == "364d88e7f9324fd5c46a893fb8dceca03c0a7714", "value of matching_2 is not correct"

assert sha1(str(type(matching_3)).encode("utf-8")+b"a0127485f3e3c1d7").hexdigest() == "49fd4d5991a0ab315c9189b72ddf47848a71e7ef", "type of matching_3 is not correct"
assert sha1(str(matching_3).encode("utf-8")+b"a0127485f3e3c1d7").hexdigest() == "ee02d9973d85b0b0e544bbe768a20ce42573b4bf", "value of matching_3 is not correct"

print('Success!')

**Question 4.3**
<br> {points: 1}

Print out 3 images and true labels from the test set that were **NOT** predicted correctly. You can reuse the `mnist_predictions_with_labels` data frame from **Question 4.2**. 

Filter for inequality between the predictions and the labels for the testing set in a data frame called `not_matching`. Afterwards, extract the row number and assign them to `not_matching_1`, `not_matching_2`, and `not_matching_3` respectively. If you need help, refer to the instructions in **Question 4.2**. 

Similar to the previous question, use the `show_digit` function we gave you above to print out the images.

In [ ]:
# Set the seed. Don't remove this!
np.random.seed(3500)

# your code here
raise NotImplementedError

In [ ]:
not_matching

In [ ]:
show_digit(not_matching_1)

In [ ]:
show_digit(not_matching_2)

In [ ]:
show_digit(not_matching_3)

In [ ]:
from hashlib import sha1
assert sha1(str(type(not_matching is None)).encode("utf-8")+b"9e9ac9c8c0719b6d").hexdigest() == "29202ec8685fbc26406487680bb9c9d47cfd627c", "type of not_matching is None is not bool. not_matching is None should be a bool"
assert sha1(str(not_matching is None).encode("utf-8")+b"9e9ac9c8c0719b6d").hexdigest() == "ddaf09ad86ed73803ac88c010913391a8b56888f", "boolean value of not_matching is None is not correct"

assert sha1(str(type(not_matching_1 is None)).encode("utf-8")+b"63ba171be103d109").hexdigest() == "1e1d68f2c4678f9bb967ac923f448d244d2b4d9f", "type of not_matching_1 is None is not bool. not_matching_1 is None should be a bool"
assert sha1(str(not_matching_1 is None).encode("utf-8")+b"63ba171be103d109").hexdigest() == "c8598a74e83f3df93932aebebebbdf66d9b88193", "boolean value of not_matching_1 is None is not correct"

assert sha1(str(type(not_matching_2 is None)).encode("utf-8")+b"3c040376dc79c90b").hexdigest() == "da544cf0e0da90e9c14c51803cd9546211c63ad6", "type of not_matching_2 is None is not bool. not_matching_2 is None should be a bool"
assert sha1(str(not_matching_2 is None).encode("utf-8")+b"3c040376dc79c90b").hexdigest() == "643f3c913b334744acc598e8b8044a5e63d57b90", "boolean value of not_matching_2 is None is not correct"

assert sha1(str(type(not_matching_3 is None)).encode("utf-8")+b"0a63d55f3e86dd46").hexdigest() == "d0f677ba5579e9e93d944709cebb95ec4314c551", "type of not_matching_3 is None is not bool. not_matching_3 is None should be a bool"
assert sha1(str(not_matching_3 is None).encode("utf-8")+b"0a63d55f3e86dd46").hexdigest() == "c5b5ee8c57972d7f114d1252513cde49b4c2b565", "boolean value of not_matching_3 is None is not correct"

assert sha1(str(type(not_matching)).encode("utf-8")+b"30baf8fa3d405a54").hexdigest() == "af96ba4e130b3af95cc3f9c71479cf993eef92e3", "type of type(not_matching) is not correct"

assert sha1(str(type(not_matching.shape)).encode("utf-8")+b"d7434389013f2e00").hexdigest() == "f424466d825bad8e3691ebbedfc059188a6a1f4e", "type of not_matching.shape is not tuple. not_matching.shape should be a tuple"
assert sha1(str(len(not_matching.shape)).encode("utf-8")+b"d7434389013f2e00").hexdigest() == "968b0ab5221cceab7cc80af692ade8553f3ec120", "length of not_matching.shape is not correct"
assert sha1(str(sorted(map(str, not_matching.shape))).encode("utf-8")+b"d7434389013f2e00").hexdigest() == "9a26de3469bcdb76ac71db0479f3ff3935011a1f", "values of not_matching.shape are not correct"
assert sha1(str(not_matching.shape).encode("utf-8")+b"d7434389013f2e00").hexdigest() == "b70b5162d97dd7557b5418956fa02d52f06e2b84", "order of elements of not_matching.shape is not correct"

assert sha1(str(type(sum(not_matching.predicted))).encode("utf-8")+b"ac5163ed5364e973").hexdigest() == "8b2126a03ec5b01190cc023be4169a882ecb21b4", "type of sum(not_matching.predicted) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(not_matching.predicted)).encode("utf-8")+b"ac5163ed5364e973").hexdigest() == "05505f1c6211d2e927fea554f47410de6213d2ae", "value of sum(not_matching.predicted) is not correct"

assert sha1(str(type(sum(not_matching.index))).encode("utf-8")+b"7e965d5b0e8b6a49").hexdigest() == "0ec5341f457f73fca58d1d64262f5496bd9cb7e6", "type of sum(not_matching.index) is not int. Please make sure it is int and not np.int64, etc. You can cast your value into an int using int()"
assert sha1(str(sum(not_matching.index)).encode("utf-8")+b"7e965d5b0e8b6a49").hexdigest() == "937c20db01e236bb6fa8543d2999ef1598c25f27", "value of sum(not_matching.index) is not correct"

assert sha1(str(type(not_matching_1)).encode("utf-8")+b"cf61980818995efb").hexdigest() == "40f7f0fb13af9ccd6bb1a5f57e663e4575097a13", "type of not_matching_1 is not correct"
assert sha1(str(not_matching_1).encode("utf-8")+b"cf61980818995efb").hexdigest() == "3007eb0f483a04effd28fce9ecd9ac1a3af69ee8", "value of not_matching_1 is not correct"

assert sha1(str(type(not_matching_2)).encode("utf-8")+b"382d23590e5be82b").hexdigest() == "23b2843b6c7b604f54e98b1ab76b1a4d913255c7", "type of not_matching_2 is not correct"
assert sha1(str(not_matching_2).encode("utf-8")+b"382d23590e5be82b").hexdigest() == "00d4c00f8b19ae978f708c2daed87bf9f25aa4c0", "value of not_matching_2 is not correct"

assert sha1(str(type(not_matching_3)).encode("utf-8")+b"b682ac11cd0b3a45").hexdigest() == "03c7b7fc16549b70aa486a266278fc4076b1489a", "type of not_matching_3 is not correct"
assert sha1(str(not_matching_3).encode("utf-8")+b"b682ac11cd0b3a45").hexdigest() == "beb4839ee1857d8609df44a629a6ae5c11280c58", "value of not_matching_3 is not correct"

print('Success!')

**Question 4.4** True or False:
<br> {points: 1}

The above images were predicted incorrectly due to messy handwriting. For example, the second image is illegible and actually looks like the letter "y".

*Assign your answer to an object called `answer4_4`. Make sure your answer is a boolean (e.g. `True` or `False`).*

In [ ]:
# your code here
raise NotImplementedError

In [ ]:
from hashlib import sha1
assert sha1(str(type(answer4_4)).encode("utf-8")+b"699734e827ba7691").hexdigest() == "f3f421c8998f1197e1ba0f7258ca477a31391c22", "type of answer4_4 is not bool. answer4_4 should be a bool"
assert sha1(str(answer4_4).encode("utf-8")+b"699734e827ba7691").hexdigest() == "cbcc6400c182d7b536f5ef5d71ccde0ae2be4883", "boolean value of answer4_4 is not correct"

print('Success!')

**Question 4.5**
<br> {points: 3}

Looking again at the plot from **Question 4.1**, what does this accuracy mean? Is it good enough that you would use this model for the Canada Post? Can you imagine a way we might improve our classifier's accuracy?

DOUBLE CLICK TO EDIT **THIS CELL** AND REPLACE THIS TEXT WITH YOUR ANSWER.